# 라이브러리 로드 및 데이터 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import pickle
import re
import json
from collections import Counter
import itertools
from datetime import datetime, timedelta
import time
import os
print(os.getcwd())

/content


# 데이터 로드 및 전처리

In [4]:
path = "/content/drive/MyDrive/issue_trends/news_data/"
ym = '202403'
df_raw_data = pd.read_csv(path+"news_{}.csv".format(ym), encoding = "utf-8-sig")
df_raw_data = df_raw_data.dropna(subset = ['content'])
df_raw_data = df_raw_data.drop_duplicates(subset = ["title"]).reset_index()

drop_text = ['[표]', '[데이터로 보는 증시]']
for i in drop_text:
    doc_boolean_tmp = list(map(lambda x: i not in x, df_raw_data.title))
    df_raw_data = df_raw_data.loc[doc_boolean_tmp, :]
df_raw_data['content'] = list(map(lambda x: re.sub("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 .]", "", x), df_raw_data['content']))

df_data = df_raw_data[df_raw_data['sid2_name']=='글로벌 경제']

In [ ]:
df_data.shape

(381, 8)

In [5]:
df_data.head(2)

,index,journal,sid1_name,sid2_name,date,title,content,href
7,7,한국경제,경제,글로벌 경제,20240331,"서방 제재에 치인 러시아, 달러 버리고 위안화 택한다",러 중앙은행 위안화 대체 통화 없어 서방 제재로 해외 자산 가치 반토막 우회로 찾으...,https://n.news.naver.com/mnews/article/015/000...
37,37,매일경제,경제,글로벌 경제,20240331,“정말 남사친 맞아?” 국민메신저 ‘친한 순서’ 줄세우기에 美10대들 발칵,청소년 대부분 사용하는 스냅챗 유료구독 달러 스냅챗 출시 친구와 거리 태양계로 확인...,https://n.news.naver.com/mnews/article/009/000...


## 1. Transformer-based Topic Modeling
- BERTopic
    - 문서의 텍스트를 임베딩 벡터로 변환하여 사용하는 토픽 모델링
    - BERT와 같은 사전 훈련된 Transformer 모델을 사용하여 문서의 임베딩을 생성한 후, 이를 사용하여 토픽을 추출
- CTM(Contextualized Topic Models
    - 사전 훈련된 Transformer 모델(BERT, RoBERTa 등)의 임베딩을 사용하여 문맥적 의미를 포함한 토픽을 추출
    - 토픽 모델링에 문맥 정보를 추가하는 방법을 제공
    -  전통적인 LDA보다 더 세밀한 토픽을 생성
- Top2Vec
    - 문서 자체에서 토픽을 자동으로 학습하는 알고리즘
    - Doc2Vec, Universal Sentence Encoder, 또는 BERT와 같은 모델을 사용하여 문서를 임베딩
- ETM(Embedding-based Topic Model)
    -  단어의 임베딩을 직접적으로 토픽 모델에 통합하는 방법
    - 단어와 토픽 사이의 관계를 임베딩 공간에서 모델링하여, 더 의미론적으로 일관된 토픽을 생성

### 1) BERTopic

In [ ]:
from bertopic import BERTopic
from transformers import BertModel, BertTokenizer

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

# 모델과 토크나이저 불러오기
model = AutoModel.from_pretrained("monologg/kobert")
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")

# 텍스트를 토큰화하고 텐서로 변환
inputs = tokenizer(list(df_data['content']), return_tensors="pt", padding=True, truncation=True)

# 임베딩을 계산
with torch.no_grad(): # 코드 블록 내의 모든 연산에서 자동 미분 기능(autograd)이 비활성화
    outputs = model(**inputs)
    # CLS 토큰의 위치에서 임베딩 추출
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()  # [batch_size, hidden_dim]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [ ]:
# BERTopic 인스턴스 생성
topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True)

# 클러스터링 실행
topics, probs = topic_model.fit_transform(embeddings)

# 결과 출력
for topic_num in set(topics):
    print(f"Topic {topic_num}: {topic_model.get_topic(topic_num)}")

In [ ]:
df_data['BERTopic'] = topics

### 2) CTM(Contextualized Topic Models)

In [ ]:
from sentence_transformers import SentenceTransformer

In [1]:
%%capture
!pip install contextualized-topic-models==2.2.0
!pip install pyldavis

# Colab에 Mecab 설치
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# %cd Mecab-ko-for-Google-Colab
# !bash install_mecab-ko_on_colab_light_220429.sh

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

mecab = Mecab()
custom_tokenizer = CustomTokenizer(mecab)

In [ ]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
train_bow_embeddings = vectorizer.fit_transform(list(df_data['content']))
print(train_bow_embeddings.shape)

vocab = vectorizer.get_feature_names_out()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens') # , device="cuda:0"
doc_embedding = model.encode(list(df_data.content))
train_contextualized_embeddings = doc_embedding

qt = TopicModelDataPreparation()
training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

In [ ]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=50, num_epochs=20)

display(ctm.fit(training_dataset))
display(ctm.get_topics(5))

In [ ]:
# 모든 문서에 대한 토픽 분포 얻기
topic_distributions = ctm.get_thetas(training_dataset)  # 문서의 토픽 분포 반환

# 각 문서에 대해 가장 높은 토픽 확률을 가진 토픽 번호를 찾기
document_topics = topic_distributions.argmax(axis=1)

# # 문서와 토픽 번호 출력 (예시)
# for i, topic_number in enumerate(document_topics):
#     print(f"Document {i} is most likely about topic {topic_number}")

In [ ]:
df_data['ctm'] = list(document_topics)
df_data.head()
df_data.to_excel(path+'ctm.xlsx')

In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

## 2. Probabilistic Topic Modeling
### 1) LDA
### 2) BTM
- 문장으로 쪼개서 해보기

## 3. Machine Learning-based Clustering
### 1) k-means
### 2) DBScan